In [19]:
!pip install torch transformers datasets pandas scikit-learn
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# incidents_rest.csv is the dataset besides the trial

# incidents_sample.csv is the trial dataset

In [ ]:
from transformers import BertTokenizer

import pandas as pd

# data = pd.read_csv('incidents_rest.csv', index_col=0)

data = pd.read_csv('incidents_train.csv', index_col=0)

data.sample()

,year,month,day,country,title,text,hazard-category,product-category,hazard,product
1192,2015,2,10,us,"2015 - Hines Nut Company, Dallas, TX Announces...","FOR IMMEDIATE RELEASE — February 10, 2015 — Hi...",biological,"nuts, nut products and seeds",salmonella,walnuts


In [ ]:
data['title_text'] = data['title'] + data['text']

In [20]:
import pandas as pd

from datasets import Dataset

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler, DataCollatorWithPadding

from torch.utils.data import DataLoader

import torch

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from tqdm.auto import tqdm

from sklearn.metrics import classification_report

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')

using device: cuda


In [ ]:
torch.device('cuda')

device(type='cuda')

In [ ]:
data.text.str.split().apply(len).describe()

count    5082.000000
mean      353.254034
std       297.662839
min        10.000000
25%       195.000000
50%       306.000000
75%       424.000000
max      8406.000000
Name: text, dtype: float64

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



def tokenize_function(examples):

    return tokenizer(examples['title_text'], padding=True, truncation=True)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Label: `Hazard Category`

* Choose your target

In [21]:
label = 'hazard-category' # change this to: 'product-category', 'hazard', 'product' to alter the ground truth

label_encoder = LabelEncoder()

data['label'] = label_encoder.fit_transform(data[label])

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))

model.to('cuda')  # Move model to GPU if available

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

* Data preprocessing

In [22]:


import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:








# Get English stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):

    word_tokens = word_tokenize(text)

    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    return ' '.join(filtered_text)



def remove_numbers(text):

    text_without_numbers = re.sub(r'\d+', '', text)

    return text_without_numbers



def lemmatize_text(text):
    # Initialize the WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    return ' '.join(lemmatized_text)


# # Apply the remove_stopwords function to the 'title' column

# data['text'] = data['text'].apply(remove_stopwords)



# # Apply the remove_numbers function to the 'title' column

# data['text'] = data['text'].apply(remove_numbers)

# # Apply lemmatization to the 'title' column
# data['text'] = data['text'].apply(lemmatize_text)

In [ ]:
# Filter the training set based on title length
# filter_data = data[(data['text'].str.split().apply(len) < 424) & (data['title'].str.split().apply(len) > 8)]
filter_data = data[data['text'].str.split().apply(len) < 424 ]

# filter_data = filter_data[filter_data['text'].str.split().apply(len) < 3424 ]

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(filter_data, test_size=0.2, random_state=42)
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Oversampling
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from torch.nn.utils.rnn import pad_sequence
import torch

# Step 2: Separate features and labels for training data
X_train = train_df['title_text']
y_train = train_df['label']

# Step 3: Tokenize the training data
X_train_tokenized = [tokenizer(title, padding=True, truncation=True, return_tensors='pt')['input_ids'][0] for title in X_train]

# Step 4: Pad the tokenized sequences to a fixed length
X_train_padded = pad_sequence(X_train_tokenized, batch_first=True, padding_value=0)

# Step 5: Convert padded sequences to a NumPy array for oversampling
X_train_np = X_train_padded.numpy()

# Step 6: Apply oversampling to the tokenized data
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train_np, y_train)

# Step 7: Create a new dataset with the oversampled data
resampled_dataset = Dataset.from_dict({'input_ids': [torch.tensor(x) for x in X_resampled], 'label': y_resampled})
resampled_dataset.set_format(type='torch', columns=['input_ids', 'label'])

# Step 8: Convert the test DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])




Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:


# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)



# Apply the tokenizer to the dataset

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)



# Create DataCollator to handle padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)



# Convert dataset to PyTorch format

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



# Create DataLoader for the oversampled training dataset
train_dataloader = DataLoader(resampled_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)

# Create DataLoader for the test dataset (remains unchanged)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

* Choose your model

In [ ]:
# Move DataLoaders to CUDA device if available
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Move DataLoaders to CUDA device if available
for batch in train_dataloader:
    for key in batch:
        batch[key] = batch[key].to(device)

for batch in test_dataloader:
    for key in batch:
        batch[key] = batch[key].to(device)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


* Train it

In [ ]:
import datetime
optimizer = AdamW(model.parameters(), lr=5e-5)



num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear",

    optimizer=optimizer,

    num_warmup_steps=0,

    num_training_steps=num_training_steps,

)



model.train()



progress_bar = tqdm(range(num_training_steps))



for epoch in range(num_epochs):

    for batch in train_dataloader:

        # batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        for key in batch:
            batch[key] = batch[key].to(device)

        outputs = model(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

current_time = datetime.datetime.now().time()
print("Finish time:", current_time)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4581 [00:00<?, ?it/s]

Finish time: 21:09:36.649196


* Assess it

In [ ]:
model.eval()

total_predictions = []

with torch.no_grad():

    for batch in test_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        # for key in batch:
        #     batch[key] = batch[key].to(device)

        outputs = model(**batch)

        predictions = torch.argmax(outputs.logits, dim=-1)

        total_predictions.extend([p.item() for p in predictions])



#print(classification_report(test_df.label, total_predictions))

predicted_labels = label_encoder.inverse_transform(total_predictions)

gold_labels = label_encoder.inverse_transform(test_df.label.values)

print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                precision    recall  f1-score   support

                     allergens       0.97      0.97      0.97       316
                    biological       1.00      0.99      0.99       212
                      chemical       0.92      0.92      0.92        49
food additives and flavourings       1.00      0.50      0.67         4
                foreign bodies       0.97      0.96      0.97        78
                         fraud       0.73      0.78      0.75        59
          organoleptic aspects       0.83      0.38      0.53        13
                  other hazard       0.64      0.73      0.68        22
              packaging defect       0.64      0.88      0.74         8

                      accuracy                           0.94       761
                     macro avg       0.86      0.79      0.80       761
                  weighted avg       0.94      0.94      0.94       761



In [ ]:
model.save_pretrained("bert_hazard_category")

# Label: `Product Category`

In [ ]:
data = pd.read_csv('incidents_train.csv', index_col=0)

In [ ]:
label = 'product-category'

label_encoder = LabelEncoder()

data['label'] = label_encoder.fit_transform(data[label])
data['title_text'] = data['title'] +data['text']

In [ ]:
# # Apply the remove_stopwords function to the 'title' column

# data['text'] = data['text'].apply(remove_stopwords)



# # Apply the remove_numbers function to the 'title' column

# data['text'] = data['text'].apply(remove_numbers)

# # Apply lemmatization to the 'title' column
# data['text'] = data['text'].apply(lemmatize_text)

In [ ]:
# Filter the training set based on title length



filter_data = data[data['text'].str.split().apply(len) <424 ]

In [ ]:


# Split the data into training and testing sets
train_df, test_df = train_test_split(filter_data, test_size=0.2, random_state=42)
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Separate features and labels for training data
X_train = train_df['title_text']
y_train = train_df['label']

# Step 3: Tokenize the training data
X_train_tokenized = [tokenizer(title, padding=True, truncation=True, return_tensors='pt')['input_ids'][0] for title in X_train]

# Step 4: Pad the tokenized sequences to a fixed length
X_train_padded = pad_sequence(X_train_tokenized, batch_first=True, padding_value=0)

# Step 5: Convert padded sequences to a NumPy array for oversampling
X_train_np = X_train_padded.numpy()

# Step 6: Apply oversampling to the tokenized data
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train_np, y_train)

# Step 7: Create a new dataset with the oversampled data
resampled_dataset = Dataset.from_dict({'input_ids': [torch.tensor(x) for x in X_resampled], 'label': y_resampled})
resampled_dataset.set_format(type='torch', columns=['input_ids', 'label'])

# Step 8: Convert the test DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])




Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:






# # Convert DataFrame to Hugging Face Dataset

train_dataset = Dataset.from_pandas(train_df)

test_dataset = Dataset.from_pandas(test_df)



# Apply the tokenizer to the dataset

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)



# Create DataCollator to handle padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)



# Convert dataset to PyTorch format

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



# Create DataLoader for the oversampled training dataset
train_dataloader = DataLoader(resampled_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)

# Create DataLoader for the test dataset (remains unchanged)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

* Train

In [ ]:
model_product_category = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))

model_product_category.to('cuda')  # Move model to GPU if available



optimizer = AdamW(model_product_category.parameters(), lr=5e-5)

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear",

    optimizer=optimizer,

    num_warmup_steps=0,

    num_training_steps=num_training_steps,

)



model_product_category.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):

    for batch in train_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        # for key in batch:
        #     batch[key] = batch[key].to(device)

        outputs = model_product_category(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)
current_time = datetime.datetime.now().time()
print("Finish time:", current_time)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/6576 [00:00<?, ?it/s]

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Finish time: 21:26:21.088741


* Test

In [ ]:
model_product_category.eval()

total_predictions = []

with torch.no_grad():

    for batch in test_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        # for key in batch:
        #     batch[key] = batch[key].to(device)

        outputs = model_product_category(**batch)

        predictions = torch.argmax(outputs.logits, dim=-1)

        total_predictions.extend([p.item() for p in predictions])



#print(classification_report(test_df.label, total_predictions, zero_division=0))

predicted_labels = label_encoder.inverse_transform(total_predictions)

gold_labels = label_encoder.inverse_transform(test_df.label.values)

print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.91      0.91      0.91        11
                      cereals and bakery products       0.82      0.76      0.79       111
     cocoa and cocoa preparations, coffee and tea       0.83      0.71      0.76        34
                                    confectionery       0.61      0.70      0.66        27
dietetic foods, food supplements, fortified foods       0.79      0.71      0.75        21
                                    fats and oils       1.00      1.00      1.00         1
                                   feed materials       0.00      0.00      0.00         5
                   food additives and flavourings       0.00      0.00      0.00         1
                           food contact materials       0.00      0.00      0.00         0
                            fruits and vegetables       0.71      0.82      0.76        7

In [ ]:
model_product_category.save_pretrained("bert_product_category")

# Label: `Hazard`

In [ ]:
data = pd.read_csv('incidents_train.csv', index_col=0)
data['title_text'] = data['title'] + data['text']

In [ ]:
label = 'hazard'

label_encoder = LabelEncoder()

data['label'] = label_encoder.fit_transform(data[label])

In [ ]:

# Filter the training set based on title length


filter_data = data[data['text'].str.split().apply(len) <424 ]

In [ ]:


# Split the data into training and testing sets
train_df, test_df = train_test_split(filter_data, test_size=0.2, random_state=42)
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Separate features and labels for training data
X_train = train_df['title_text']
y_train = train_df['label']

# Step 3: Tokenize the training data
X_train_tokenized = [tokenizer(title, padding=True, truncation=True, return_tensors='pt')['input_ids'][0] for title in X_train]

# Step 4: Pad the tokenized sequences to a fixed length
X_train_padded = pad_sequence(X_train_tokenized, batch_first=True, padding_value=0)

# Step 5: Convert padded sequences to a NumPy array for oversampling
X_train_np = X_train_padded.numpy()

# Step 6: Apply oversampling to the tokenized data
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train_np, y_train)

# Step 7: Create a new dataset with the oversampled data
resampled_dataset = Dataset.from_dict({'input_ids': [torch.tensor(x) for x in X_resampled], 'label': y_resampled})
resampled_dataset.set_format(type='torch', columns=['input_ids', 'label'])

# Step 8: Convert the test DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:






# Convert DataFrame to Hugging Face Dataset

train_dataset = Dataset.from_pandas(train_df)

test_dataset = Dataset.from_pandas(test_df)



# Apply the tokenizer to the dataset

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)



# Create DataCollator to handle padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)



# Convert dataset to PyTorch format

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



# Create DataLoader for the oversampled training dataset
train_dataloader = DataLoader(resampled_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)

# Create DataLoader for the test dataset (remains unchanged)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:
model_hazard = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))

model_hazard.to('cuda')  # Move model to GPU if available



optimizer = AdamW(model_hazard.parameters(), lr=5e-5)



num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear",

    optimizer=optimizer,

    num_warmup_steps=0,

    num_training_steps=num_training_steps,

)



model_hazard.train()



progress_bar = tqdm(range(num_training_steps))



for epoch in range(num_epochs):

    for batch in train_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available

        outputs = model_hazard(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/17580 [00:00<?, ?it/s]

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
model_hazard.eval()

total_predictions = []

with torch.no_grad():

    for batch in test_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available

        outputs = model_hazard(**batch)

        predictions = torch.argmax(outputs.logits, dim=-1)

        total_predictions.extend([p.item() for p in predictions])



#print(classification_report(test_df.label, total_predictions, zero_division=0))

predicted_labels = label_encoder.inverse_transform(total_predictions)

gold_labels = label_encoder.inverse_transform(test_df.label.values)

print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                 precision    recall  f1-score   support

                                      Aflatoxin       1.00      1.00      1.00         1
                                 abnormal smell       0.00      0.00      0.00         2
                                alcohol content       0.00      0.00      0.00         0
                                      alkaloids       0.00      0.00      0.00         2
                                      allergens       0.00      0.00      0.00         3
                                         almond       0.88      1.00      0.94        15
                                  bacillus spp.       0.50      0.67      0.57         3
                           bad smell / off odor       0.00      0.00      0.00         3
                                  bone fragment       1.00      1.00      1.00         1
                                     brazil nut       0.00      0.00      0.00         1
                    

In [ ]:
model_hazard.save_pretrained("bert_hazard")

# Label: `product`

In [ ]:
data = pd.read_csv('incidents_train.csv', index_col=0)

data['title_text'] = data['title'] + data['text']


In [23]:
label = 'product'

label_encoder = LabelEncoder()

data['label'] = label_encoder.fit_transform(data[label])

In [ ]:
# Filter the training set based on title length


filter_data = data[data['text'].str.split().apply(len) <424 ]



In [ ]:

# Split the data into training and testing sets
train_df, test_df = train_test_split(filter_data, test_size=0.2, random_state=42)
# train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Separate features and labels for training data
X_train = train_df['title_text']
y_train = train_df['label']

# Step 3: Tokenize the training data
X_train_tokenized = [tokenizer(title, padding=True, truncation=True, return_tensors='pt')['input_ids'][0] for title in X_train]

# Step 4: Pad the tokenized sequences to a fixed length
X_train_padded = pad_sequence(X_train_tokenized, batch_first=True, padding_value=0)

# Step 5: Convert padded sequences to a NumPy array for oversampling
X_train_np = X_train_padded.numpy()

# Step 6: Apply oversampling to the tokenized data
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train_np, y_train)

# Step 7: Create a new dataset with the oversampled data
resampled_dataset = Dataset.from_dict({'input_ids': [torch.tensor(x) for x in X_resampled], 'label': y_resampled})
resampled_dataset.set_format(type='torch', columns=['input_ids', 'label'])

# Step 8: Convert the test DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:





# Convert DataFrame to Hugging Face Dataset

train_dataset = Dataset.from_pandas(train_df)

test_dataset = Dataset.from_pandas(test_df)



# Apply the tokenizer to the dataset

train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)



# Create DataCollator to handle padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)



# Convert dataset to PyTorch format

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])




# Create DataLoader for the oversampled training dataset
train_dataloader = DataLoader(resampled_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)

# Create DataLoader for the test dataset (remains unchanged)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

Map:   0%|          | 0/761 [00:00<?, ? examples/s]

In [ ]:
model_product = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))

model_product.to('cuda')  # Move model to GPU if available



optimizer = AdamW(model_product.parameters(), lr=5e-5)

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear",

    optimizer=optimizer,

    num_warmup_steps=0,

    num_training_steps=num_training_steps,

)

model_product.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):

    for batch in train_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available

        outputs = model_product(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/38436 [00:00<?, ?it/s]

C:\Users\lzh\.conda\envs\DeepLearning\Lib\site-packages\transformers\tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
model_product.eval()

total_predictions = []

with torch.no_grad():

    for batch in test_dataloader:

        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available

        outputs = model_product(**batch)

        predictions = torch.argmax(outputs.logits, dim=-1)

        total_predictions.extend([p.item() for p in predictions])



#print(classification_report(test_df.label, total_predictions, zero_division=0))

predicted_labels = label_encoder.inverse_transform(total_predictions)

gold_labels = label_encoder.inverse_transform(test_df.label.values)

print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       0.67      1.00      0.80         2
                                                 Fishes not identified       0.38      0.60      0.46         5
                                              Not classified pork meat       0.00      0.00      0.00         0
                                            Pangas catfishes (generic)       0.00      0.00      0.00         0
                                   Precooked cooked pork meat products       0.00      0.00      0.00         6
                                                   alcoholic beverages       0.00      0.00      0.00         1
                                                       alfalfa sprouts       0.00      0.00      0.00         5
                                                                 algae       0.00      0.00      0.00  

In [ ]:
model_product.save_pretrained("bert_product")

tokenizer.save_pretrained("bert_tokenizer")

In [ ]:
!zip bert_baseline.zip bert_*

# Loading a trained baseline

In [ ]:
import torch

from transformers import BertTokenizer, BertForSequenceClassification



def predict(texts, model_path, tokenizer_path="bert_tokenizer"):

    # Load the saved tokenizer

    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)



    # Load the saved model

    model = BertForSequenceClassification.from_pretrained(model_path)



    # Move model to GPU if available

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)



    # Tokenize the input texts

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")



    # Move inputs to the same device as the model

    inputs = {key: value.to(device) for key, value in inputs.items()}



    # Set the model to evaluation mode

    model.eval()



    # Make predictions

    with torch.no_grad():

        outputs = model(**inputs)

        logits = outputs.logits

        predictions = torch.argmax(logits, dim=-1)



    return predictions

In [ ]:
from sklearn.metrics import f1_score



def compute_score(hazards_true, products_true, hazards_pred, products_pred):

  # compute f1 for hazards:

  f1_hazards = f1_score(

    hazards_true,

    hazards_pred,

    average='macro'

  )



  # compute f1 for products:

  f1_products = f1_score(

    products_true[hazards_pred == hazards_true],

    products_pred[hazards_pred == hazards_true],

    average='macro'

  )



  return (f1_hazards + f1_products) / 2.

### Sub-Task 1:

In [24]:
predictions = predict(trial.title.to_list(), "bert_hazard_category")



# Decode predictions back to string labels

label_encoder = LabelEncoder()

label_encoder.fit(data['hazard-category'])



trial['bert-hazard-category'] = predict(trial.title.to_list(), "bert_hazard_category")

print(classification_report(trial['hazard-category'], trial['bert-hazard-category'], zero_division=0))

NameError: name 'trial' is not defined

In [ ]:
predictions = predict(trial.title.to_list(), "bert_product_category")



# Decode predictions back to string labels

label_encoder = LabelEncoder()

label_encoder.fit(data['product-category'])



trial['bert-product-category'] = predict(trial.title.to_list(), "bert_product_category")

print(classification_report(trial['product-category'], trial['bert-product-category'], zero_division=0))

In [ ]:
print('Score Sub-Task 1:', compute_score(trial['hazard-category'], trial['product-category'], trial['bert-hazard-category'], trial['bert-product-category']))

### Sub-Task 2:

In [ ]:
predictions = predict(trial.title.to_list(), "bert_hazard")



# Decode predictions back to string labels

label_encoder = LabelEncoder()

label_encoder.fit(data['hazard'])



trial['bert-hazard'] = predict(trial.title.to_list(), "bert_hazard")

print(classification_report(trial['hazard'], trial['bert-hazard'], zero_division=0))

In [ ]:
predictions = predict(trial.title.to_list(), "bert_product")



# Decode predictions back to string labels

label_encoder = LabelEncoder()

label_encoder.fit(data['product'])



trial['bert-product'] = predict(trial.title.to_list(), "bert_product")

print(classification_report(trial['product'], trial['bert-product'], zero_division=0))

In [ ]:
print('Score Sub-Task 2:', compute_score(trial['hazard'], trial['product'], trial['bert-hazard'], trial['bert-product']))

Inference Step

In [ ]:
import pandas as pd

# Load the labeled dataset
data = pd.read_csv('/path_to_your_dataset/incidents_valid.csv')

# Preview the dataset
print(data.head())


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the checkpoint
checkpoint_path = "/path_to_your_checkpoint"  # Replace with the checkpoint folder path
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
def predict(texts, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.cpu().numpy()

# Example predictions
texts = data['title'] + " " + data['text']  # Combine relevant columns if necessary
predictions = predict(texts.tolist(), model, tokenizer, device)


In [ ]:
from sklearn.metrics import classification_report

# Convert true labels to integers using LabelEncoder if necessary
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(data['hazard-category'])  # Change column name accordingly

# Evaluate predictions
print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))


In [1]:
import pandas as pd

# Load the test dataset
test_data_path = "/content/incidents_valid.csv"
test_data = pd.read_csv(test_data_path)

# Preview the dataset
print(test_data.head())


   Unnamed: 0  year  month  day country  \
0           0  1994      5    5      us   
1           1  1994      5   12      us   
2           2  1995      4   16      us   
3           3  1998      7   16      ca   
4           4  1998      8    6      us   

                                               title  \
0                   Recall Notification: FSIS-017-94   
1                   Recall Notification: FSIS-048-94   
2                   Recall Notification: FSIS-032-95   
3  Archive - ALLERGY ALERT -- PRESENCE OF UNDECLA...   
4                   Recall Notification: FSIS-018-98   

                                                text hazard-category  \
0  Case Number: 017-94   \n            Date Opene...      biological   
1  Case Number: 048-94   \n            Date Opene...      biological   
2  Case Number: 032-95   \n            Date Opene...      biological   
3  PRESENCE OF UNDECLARED NUTS IN ORIGINALE AUGUS...       allergens   
4  Case Number: 018-98  Recall Notification 

In [10]:
from google.colab import drive
drive.mount('/content/drive')

product_category_checkpoint_path = "/content/drive/My Drive/checkpoint/product-category/product-category-1"
hazard_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard/hazard-3"
hazard_category_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard-category/hazard-category-2"
product_checkpoint_path = "/content/drive/My Drive/checkpoint/product/product-2"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import torch
# import pandas as pd

# # Paths to checkpoints
# product_category_checkpoint_path = "/content/drive/My Drive/checkpoint/product-category/product-category-1"
# hazard_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard/hazard-3"
# hazard_category_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard-category/hazard-category-2"
# product_checkpoint_path = "/content/drive/My Drive/checkpoint/product/product-2"

# # Function to load model and tokenizer
# def load_model_and_tokenizer(checkpoint_path, base_model_name="bert-base-uncased"):
#     # Replace 'bert-base-uncased' with the correct base model used in training
#     model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, local_files_only=True)
#     tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     return model, tokenizer, device

# # Load models and tokenizers for all tasks
# product_category_model, product_category_tokenizer, product_category_device = load_model_and_tokenizer(product_category_checkpoint_path)
# hazard_model, hazard_tokenizer, hazard_device = load_model_and_tokenizer(hazard_checkpoint_path)
# hazard_category_model, hazard_category_tokenizer, hazard_category_device = load_model_and_tokenizer(hazard_category_checkpoint_path)
# product_model, product_tokenizer, product_device = load_model_and_tokenizer(product_checkpoint_path)

# # Function for inference
# def predict(texts, model, tokenizer, device):
#     model.eval()
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1)
#     return predictions.cpu().numpy()

# # Load the test dataset
# test_data_path = "/content/incidents_valid.csv"
# data = pd.read_csv(test_data_path)

# # Generate predictions for each task
# data['product_category_pred'] = predict(data['text'].tolist(), product_category_model, product_category_tokenizer, product_category_device)
# data['hazard_pred'] = predict(data['text'].tolist(), hazard_model, hazard_tokenizer, hazard_device)
# data['hazard_category_pred'] = predict(data['text'].tolist(), hazard_category_model, hazard_category_tokenizer, hazard_category_device)
# data['product_pred'] = predict(data['text'].tolist(), product_model, product_tokenizer, product_device)

# # Save the predictions to a CSV file
# output_path = "/content/predictions.csv"
# data.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")


Predictions saved to /content/predictions.csv


In [13]:
# # Load id2label mapping for each model
# def load_id2label(config_path):
#     with open(config_path, 'r') as f:
#         config = json.load(f)
#     return config.get('id2label', {})

# # Paths to config.json files
# product_category_config_path = "/content/drive/My Drive/checkpoint/product-category/product-category-1/config.json"
# hazard_config_path = "/content/drive/My Drive/checkpoint/hazard/hazard-3/config.json"
# hazard_category_config_path = "/content/drive/My Drive/checkpoint/hazard-category/hazard-category-2/config.json"
# product_config_path = "/content/drive/My Drive/checkpoint/product/product-2/config.json"

# # Load mappings
# product_category_id2label = load_id2label(product_category_config_path)
# hazard_id2label = load_id2label(hazard_config_path)
# hazard_category_id2label = load_id2label(hazard_category_config_path)
# product_id2label = load_id2label(product_config_path)

# # Modify the prediction function to map numeric predictions to labels
# def predict_with_labels(texts, model, tokenizer, device, id2label):
#     model.eval()
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
#     # Map predictions to labels
#     return [id2label.get(str(pred), pred) for pred in predictions]

# # Generate predictions and map them to labels
# data['product_category_pred'] = predict_with_labels(data['text'].tolist(), product_category_model, product_category_tokenizer, product_category_device, product_category_id2label)
# data['hazard_pred'] = predict_with_labels(data['text'].tolist(), hazard_model, hazard_tokenizer, hazard_device, hazard_id2label)
# data['hazard_category_pred'] = predict_with_labels(data['text'].tolist(), hazard_category_model, hazard_category_tokenizer, hazard_category_device, hazard_category_id2label)
# data['product_pred'] = predict_with_labels(data['text'].tolist(), product_model, product_tokenizer, product_device, product_id2label)

# # Save the updated predictions to CSV
# output_path = "/content/predictions_with_labels.csv"
# data.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")


Predictions saved to /content/predictions_with_labels.csv


In [17]:
# import json

# # Function to load id2label mapping from config.json
# def load_id2label(config_path):
#     with open(config_path, 'r') as f:
#         config = json.load(f)
#     return config.get('id2label', {})

# # Paths to config.json files for each task
# product_category_config_path = "/content/drive/My Drive/checkpoint/product-category/product-category-1/config.json"
# hazard_config_path = "/content/drive/My Drive/checkpoint/hazard/hazard-3/config.json"
# hazard_category_config_path = "/content/drive/My Drive/checkpoint/hazard-category/hazard-category-2/config.json"
# product_config_path = "/content/drive/My Drive/checkpoint/product/product-2/config.json"

# # Load id2label mappings
# product_category_id2label = load_id2label(product_category_config_path)
# hazard_id2label = load_id2label(hazard_config_path)
# hazard_category_id2label = load_id2label(hazard_category_config_path)
# product_id2label = load_id2label(product_config_path)

# # Function to map predictions to labels
# def map_predictions_to_labels(predictions, id2label):
#     return [id2label.get(f"LABEL_{int(pred)}", "UNKNOWN") for pred in predictions]

# # Update predictions in the inference process
# data['product_category_pred'] = map_predictions_to_labels(
#     predict(data['text'].tolist(), product_category_model, product_category_tokenizer, product_category_device),
#     product_category_id2label
# )

# data['hazard_pred'] = map_predictions_to_labels(
#     predict(data['text'].tolist(), hazard_model, hazard_tokenizer, hazard_device),
#     hazard_id2label
# )

# data['hazard_category_pred'] = map_predictions_to_labels(
#     predict(data['text'].tolist(), hazard_category_model, hazard_category_tokenizer, hazard_category_device),
#     hazard_category_id2label
# )

# data['product_pred'] = map_predictions_to_labels(
#     predict(data['text'].tolist(), product_model, product_tokenizer, product_device),
#     product_id2label
# )

# # Save the predictions to a CSV file
# output_path = "/content/predictions.csv"
# data.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")


Predictions saved to /content/predictions.csv


In [26]:
# import pandas as pd
# import pickle
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import torch

# # Paths to checkpoints
# product_category_checkpoint_path = "/content/drive/My Drive/checkpoint/product-category/product-category-1"
# hazard_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard/hazard-3"
# hazard_category_checkpoint_path = "/content/drive/My Drive/checkpoint/hazard-category/hazard-category-2"
# product_checkpoint_path = "/content/drive/My Drive/checkpoint/product/product-2"

# # Path to the label encoder
# label_encoder_path = "label_encoder.pkl"

# # Load the LabelEncoder
# with open(label_encoder_path, 'rb') as f:
#     label_encoder = pickle.load(f)

# # Function to load model and tokenizer
# def load_model_and_tokenizer(checkpoint_path, base_model_name="bert-base-uncased"):
#     model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, local_files_only=True)
#     tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     return model, tokenizer, device

# # Load models and tokenizers for all tasks
# product_category_model, product_category_tokenizer, product_category_device = load_model_and_tokenizer(product_category_checkpoint_path)
# hazard_model, hazard_tokenizer, hazard_device = load_model_and_tokenizer(hazard_checkpoint_path)
# hazard_category_model, hazard_category_tokenizer, hazard_category_device = load_model_and_tokenizer(hazard_category_checkpoint_path)
# product_model, product_tokenizer, product_device = load_model_and_tokenizer(product_checkpoint_path)

# # Function for inference
# def predict(texts, model, tokenizer, device):
#     model.eval()
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
#     return predictions

# # Load the test dataset
# test_data_path = "/content/incidents_valid.csv"
# data = pd.read_csv(test_data_path)

# # Generate predictions for each task and decode them
# data['product_category_pred'] = label_encoder.inverse_transform(
#     predict(data['text'].tolist(), product_category_model, product_category_tokenizer, product_category_device)
# )
# data['hazard_pred'] = label_encoder.inverse_transform(
#     predict(data['text'].tolist(), hazard_model, hazard_tokenizer, hazard_device)
# )
# data['hazard_category_pred'] = label_encoder.inverse_transform(
#     predict(data['text'].tolist(), hazard_category_model, hazard_category_tokenizer, hazard_category_device)
# )
# data['product_pred'] = label_encoder.inverse_transform(
#     predict(data['text'].tolist(), product_model, product_tokenizer, product_device)
# )

# # Save the predictions to a CSV file
# output_path = "/content/predictions_with_labels.csv"
# data.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'label_encoder.pkl'

In [28]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# import pickle

# # Load the training dataset
# train_data_path = "incidents_train.csv"
# train_data = pd.read_csv(train_data_path)

# # List of target columns
# target_columns = ['hazard-category', 'product-category', 'hazard', 'product']

# # Create and save LabelEncoders for each target column
# for column in target_columns:
#     label_encoder = LabelEncoder()
#     train_data[f"{column}_encoded"] = label_encoder.fit_transform(train_data[column])

#     # Save the LabelEncoder
#     with open(f'label_encoder_{column}.pkl', 'wb') as f:
#         pickle.dump(label_encoder, f)

#     print(f"Saved LabelEncoder for {column}. Classes: {label_encoder.classes_}")


Saved LabelEncoder for hazard-category. Classes: ['allergens' 'biological' 'chemical' 'food additives and flavourings'
 'foreign bodies' 'fraud' 'migration' 'organoleptic aspects'
 'other hazard' 'packaging defect']
Saved LabelEncoder for product-category. Classes: ['alcoholic beverages' 'cereals and bakery products'
 'cocoa and cocoa preparations, coffee and tea' 'confectionery'
 'dietetic foods, food supplements, fortified foods' 'fats and oils'
 'feed materials' 'food additives and flavourings'
 'food contact materials' 'fruits and vegetables' 'herbs and spices'
 'honey and royal jelly' 'ices and desserts'
 'meat, egg and dairy products' 'non-alcoholic beverages'
 'nuts, nut products and seeds' 'other food product / mixed' 'pet feed'
 'prepared dishes and snacks' 'seafood'
 'soups, broths, sauces and condiments' 'sugars and syrups']
Saved LabelEncoder for hazard. Classes: ['Aflatoxin' 'abnormal smell' 'alcohol content' 'alkaloids' 'allergens'
 'almond' 'altered organoleptic characte

In [31]:
# import pandas as pd
# import numpy as np
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Define the function to load models and tokenizers
# def load_model_and_tokenizer(checkpoint_path, base_model_name="bert-base-uncased"):
#     model = AutoModelForSequenceClassification.from_pretrained(
#         checkpoint_path, local_files_only=True
#     )
#     tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     model.to(device)
#     return model, tokenizer, device

# # Define the function to make predictions
# def predict(texts, model, tokenizer, device):
#     model.eval()
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
#     return predictions

# # Define a function to handle unseen labels safely
# def safe_inverse_transform(label_encoder, predictions_encoded):
#     # Get valid classes from the label encoder
#     valid_classes = set(range(len(label_encoder.classes_)))
#     # Replace unseen labels with a default value
#     predictions_safe = np.array([
#         pred if pred in valid_classes else -1
#         for pred in predictions_encoded
#     ])
#     # Add an "unknown" label to the label encoder if not present
#     if "UNKNOWN" not in label_encoder.classes_:
#         label_encoder.classes_ = np.append(label_encoder.classes_, "UNKNOWN")
#     # Transform safely
#     return np.array([
#         label_encoder.classes_[pred] if pred != -1 else "UNKNOWN"
#         for pred in predictions_safe
#     ])

# # Load the training dataset to fit LabelEncoders
# train_data_path = "/content/incidents_train.csv"
# train_data = pd.read_csv(train_data_path)

# # Initialize LabelEncoders for each target column
# from sklearn.preprocessing import LabelEncoder

# label_encoders = {}
# target_columns = ["product-category", "hazard", "hazard-category", "product"]
# for column in target_columns:
#     label_encoders[column] = LabelEncoder()
#     label_encoders[column].fit(train_data[column])

# # Load models, tokenizers, and devices
# product_category_model, product_category_tokenizer, product_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product-category/product-category-1")
# hazard_model, hazard_tokenizer, hazard_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard/hazard-3")
# hazard_category_model, hazard_category_tokenizer, hazard_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard-category/hazard-category-2")
# product_model, product_tokenizer, product_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product/product-2")

# # Load the test dataset
# test_data_path = "/content/incidents_valid.csv"
# data = pd.read_csv(test_data_path)

# # Generate predictions for each task and decode them
# for column, model, tokenizer, device in zip(
#     target_columns,
#     [product_category_model, hazard_model, hazard_category_model, product_model],
#     [product_category_tokenizer, hazard_tokenizer, hazard_category_tokenizer, product_tokenizer],
#     [product_category_device, hazard_device, hazard_category_device, product_device]
# ):
#     predictions_encoded = predict(data['text'].tolist(), model, tokenizer, device)
#     data[f"{column}_pred"] = safe_inverse_transform(label_encoders[column], predictions_encoded)

# # Save the predictions to a CSV file
# output_path = "/content/predictions_with_labels.csv"
# data.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")



Predictions saved to /content/predictions_with_labels.csv


In [32]:
# import pandas as pd
# import numpy as np
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import torch
# from sklearn.preprocessing import LabelEncoder

# # Define the function to load models and tokenizers
# def load_model_and_tokenizer(checkpoint_path, base_model_name="bert-base-uncased"):
#     model = AutoModelForSequenceClassification.from_pretrained(
#         checkpoint_path, local_files_only=True
#     )
#     tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     model.to(device)
#     return model, tokenizer, device

# # Define the function to make predictions
# def predict(texts, model, tokenizer, device):
#     model.eval()
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
#     return predictions

# # Define a function to handle unseen labels safely
# def safe_inverse_transform(label_encoder, predictions_encoded):
#     # Get valid classes from the label encoder
#     valid_classes = set(range(len(label_encoder.classes_)))
#     # Replace unseen labels with a default value
#     predictions_safe = np.array([
#         pred if pred in valid_classes else -1
#         for pred in predictions_encoded
#     ])
#     # Add an "unknown" label to the label encoder if not present
#     if "UNKNOWN" not in label_encoder.classes_:
#         label_encoder.classes_ = np.append(label_encoder.classes_, "UNKNOWN")
#     # Transform safely
#     return np.array([
#         label_encoder.classes_[pred] if pred != -1 else "UNKNOWN"
#         for pred in predictions_safe
#     ])

# # Load the training dataset to fit LabelEncoders
# train_data_path = "/content/incidents_train.csv"
# train_data = pd.read_csv(train_data_path)

# # Initialize LabelEncoders for each target column
# label_encoders = {}
# target_columns = ["product-category", "hazard", "hazard-category", "product"]
# for column in target_columns:
#     label_encoders[column] = LabelEncoder()
#     label_encoders[column].fit(train_data[column])

# # Load models, tokenizers, and devices
# product_category_model, product_category_tokenizer, product_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product-category/product-category-1")
# hazard_model, hazard_tokenizer, hazard_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard/hazard-3")
# hazard_category_model, hazard_category_tokenizer, hazard_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard-category/hazard-category-2")
# product_model, product_tokenizer, product_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product/product-2")

# # Load the test dataset
# test_data_path = "/content/incidents_valid.csv"
# data = pd.read_csv(test_data_path)

# # Generate predictions for each task and decode them
# for column, model, tokenizer, device in zip(
#     target_columns,
#     [product_category_model, hazard_model, hazard_category_model, product_model],
#     [product_category_tokenizer, hazard_tokenizer, hazard_category_tokenizer, product_tokenizer],
#     [product_category_device, hazard_device, hazard_category_device, product_device]
# ):
#     predictions_encoded = predict(data['text'].tolist(), model, tokenizer, device)
#     data[f"{column}_pred"] = safe_inverse_transform(label_encoders[column], predictions_encoded)

# # Select only the required columns for submission
# submission = data[["product-category_pred", "hazard_pred", "hazard-category_pred", "product_pred"]]
# submission.columns = ["product-category", "hazard", "hazard-category", "product"]

# # Save the submission file
# submission_path = "/content/submission.csv"
# submission.to_csv(submission_path, index=False)
# print(f"Submission file saved to {submission_path}")

# # Compress the submission file into ZIP format
# import zipfile

# zip_path = "/content/submission.zip"
# with zipfile.ZipFile(zip_path, "w") as zipf:
#     zipf.write(submission_path, arcname="submission.csv")

# print(f"Compressed submission file saved to {zip_path}")


Submission file saved to /content/submission.csv
Compressed submission file saved to /content/submission.zip


In [38]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
import torch
import zipfile

# Define the function to load models and tokenizers
def load_model_and_tokenizer(checkpoint_path, base_model_name="bert-base-uncased"):
    model = AutoModelForSequenceClassification.from_pretrained(
        checkpoint_path, local_files_only=True
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return model, tokenizer, device

# Define the function to make predictions
def predict(texts, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
    return predictions

# Define a function to handle unseen labels safely
def safe_inverse_transform(label_encoder, predictions_encoded):
    # Get valid classes from the label encoder
    valid_classes = set(range(len(label_encoder.classes_)))
    # Replace unseen labels with a default value
    predictions_safe = np.array([
        pred if pred in valid_classes else -1
        for pred in predictions_encoded
    ])
    # Add an "unknown" label to the label encoder if not present
    if "UNKNOWN" not in label_encoder.classes_:
        label_encoder.classes_ = np.append(label_encoder.classes_, "UNKNOWN")
    # Transform safely
    return np.array([
        label_encoder.classes_[pred] if pred != -1 else "UNKNOWN"
        for pred in predictions_safe
    ])

# Load the training dataset to fit LabelEncoders
train_data_path = "/content/incidents_train.csv"
train_data = pd.read_csv(train_data_path)

# Initialize LabelEncoders for each target column
label_encoders = {}
target_columns = ["product-category", "hazard", "hazard-category", "product"]
for column in target_columns:
    label_encoders[column] = LabelEncoder()
    label_encoders[column].fit(train_data[column])

# Load models, tokenizers, and devices
product_category_model, product_category_tokenizer, product_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product-category/product-category-1")
hazard_model, hazard_tokenizer, hazard_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard/hazard-3")
hazard_category_model, hazard_category_tokenizer, hazard_category_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/hazard-category/hazard-category-2")
product_model, product_tokenizer, product_device = load_model_and_tokenizer("/content/drive/MyDrive/checkpoint/product/product-2")

# Load the updated test dataset
test_data_path = "/content/incidents 2.csv"
data = pd.read_csv(test_data_path)

# Ensure no NaN values in the text column
data["text"] = data["text"].fillna("")

# Generate predictions for each task and decode them
for column, model, tokenizer, device in zip(
    target_columns,
    [product_category_model, hazard_model, hazard_category_model, product_model],
    [product_category_tokenizer, hazard_tokenizer, hazard_category_tokenizer, product_tokenizer],
    [product_category_device, hazard_device, hazard_category_device, product_device]
):
    predictions_encoded = predict(data['text'].tolist(), model, tokenizer, device)
    # Align predictions with test data rows
    predictions_encoded = np.pad(
        predictions_encoded,
        (0, len(data) - len(predictions_encoded)),
        constant_values=-1
    )
    data[f"{column}_pred"] = safe_inverse_transform(label_encoders[column], predictions_encoded)

# Verify the number of rows in the predictions
assert len(data) == len(pd.read_csv(test_data_path)), "Mismatch in row counts between test data and predictions."

# Select only the required columns for submission
submission_data = data[["hazard-category_pred", "product-category_pred", "hazard_pred", "product_pred"]].rename(
    columns={
        "hazard-category_pred": "hazard-category",
        "product-category_pred": "product-category",
        "hazard_pred": "hazard",
        "product_pred": "product"
    }
)

# Save the predictions to a CSV file
submission_path = "/content/submission.csv"
submission_data.to_csv(submission_path, index=False)
print(f"Submission file saved to {submission_path}")

# Compress the submission file into a ZIP file
zip_path = "/content/submission.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write(submission_path, arcname="submission.csv")
print(f"Submission zip file saved to {zip_path}")



Submission file saved to /content/submission.csv
Submission zip file saved to /content/submission.zip
